In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
import gc
import random
os.getcwd()

# Clusters used: Home Focused, Spending Fanatics, Holiday Shopper, Urban Affluence, Rising Hispanic

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/Experian_Control_Test'

In [2]:
Q4_control_segments_files=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/by_group_quadrant_24/C_*")
len(Q4_control_segments_files)

36

In [3]:
Q4_control_segments_files[3]

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_T_H_2019Q1.csv'

In [4]:
df_control=pd.DataFrame()

for file_path in Q4_control_segments_files:
    df=pd.read_csv(file_path,dtype=str)
    df_control=df_control.append(df)
    
print(df_control.shape)
email_control_set=set(df_control['email_address_hash'].unique().tolist())
print(len(email_control_set))
df_control.head(4)

(499998, 3)
499998


,customer_id_hashed,email_address_hash,segment
0,6b04ce2c2b1b832835f1afa3ee7b2886f47da2d3101dac...,122c13aa1d02032e77f2d3b8719740d70f8abeb8ea9437...,C_Quadrant III_P_H_2019Q1
1,1894193341d61e377f8e9e5b8feeb49bacb7d5cffc2ff8...,25ce63822e1b6d2c51e773f062322dee8d62b1fdf35170...,C_Quadrant III_P_H_2019Q1
2,75544ed3ca318ca39cec4f534d579eea5eb532ca9a7983...,2a210f4963167fb0bf69386b71b6e2ce6c05151cb6440f...,C_Quadrant III_P_H_2019Q1
3,872b386e035813bb595cda0f0bbb11aa11d3f8f1a0fd9f...,1253e35da3454a6edd02736e4dbe56aad61ccfd8434001...,C_Quadrant III_P_H_2019Q1


In [5]:
Eperian_HH=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/MS_Experian_HH_Full.zip",compression="zip",dtype=str,sep="|",usecols=['Customer_Num','Email_Address_hash','multi_cluster_desc'])
Eperian_GEO=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/MS_Experian_GEO_Full.zip",compression="zip",dtype=str,sep="|",usecols=['Customer_Num','Email_Address_hash','multi_cluster_desc'])
Eperian=Eperian_HH.append(Eperian_GEO)

del Eperian_HH
del Eperian_GEO
gc.collect()

Eperian=Eperian[Eperian['multi_cluster_desc'].isin(['Home Focused','Spending Fanatics','Holiday Shoppers','Urban Affluence','Rising Hispanics'])]


In [6]:
print(Eperian.shape)

(8201477, 3)


In [7]:
Eperian_in_Q4_Control=Eperian[Eperian['Email_Address_hash'].isin(email_control_set)]
print(Eperian_in_Q4_Control.shape)
Eperian_not_in_Q4_Control=Eperian[~Eperian['Email_Address_hash'].isin(email_control_set)]
print(Eperian_not_in_Q4_Control.shape)

(202546, 3)
(7998931, 3)


In [8]:
all_control_df=pd.DataFrame()
all_test_df=pd.DataFrame()
random.seed(1)

for multi_cluster_desc,group in Eperian_not_in_Q4_Control.groupby(['multi_cluster_desc']):
    random_list=random.sample(range(len(group)), int(np.ceil(len(group)*0.05))) # Changed to 5%
    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    
    control_df=group[group['index'].isin(random_list)]
    test_df=group[~group['index'].isin(random_list)]
    
    all_control_df=all_control_df.append(control_df)
    all_test_df=all_test_df.append(test_df)

In [9]:
del all_control_df['index']
del all_test_df['index']

all_control_df=all_control_df.reset_index()
all_test_df=all_test_df.reset_index()

del all_control_df['index']
del all_test_df['index']

all_control_df['segment']="Experian_"+"C_"+all_control_df['multi_cluster_desc']
all_test_df['segment']="Experian_"+"T_"+all_test_df['multi_cluster_desc']
all_df_with_segment=all_control_df.append(all_test_df)

In [10]:
all_df_with_segment.head(2)

,Customer_Num,Email_Address_hash,multi_cluster_desc,segment
0,4945660,23ff9e2ec16a0a407454533418cd32f8e4b1ac815ae754...,Holiday Shoppers,Experian_C_Holiday Shoppers
1,46271575,f89762653a24886c789413548159d195428ea64d520256...,Holiday Shoppers,Experian_C_Holiday Shoppers


In [11]:
import hashlib
all_df_with_segment['customer_id_hashed']=all_df_with_segment['Customer_Num'].apply(lambda x: hashlib.sha256(x.encode('UTF-8')).hexdigest())


In [12]:
writer_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/Experian_Control_Test/output/"

In [13]:
for segment,group in all_df_with_segment.groupby(['segment']):
    group=group[['customer_id_hashed','Email_Address_hash','segment']]
    group.to_csv(writer_folder+segment+".csv",index=False)
    print(segment,len(group))

Experian_C_Holiday Shoppers 45109
Experian_C_Home Focused 108612
Experian_C_Rising Hispanics 110393
Experian_C_Spending Fanatics 47445
Experian_C_Urban Affluence 88390
Experian_T_Holiday Shoppers 857058
Experian_T_Home Focused 2063612
Experian_T_Rising Hispanics 2097462
Experian_T_Spending Fanatics 901449
Experian_T_Urban Affluence 1679401


In [14]:
summary_count=all_df_with_segment.groupby('segment')['Email_Address_hash'].count().to_frame().reset_index().rename(columns={"Email_Address_hash":"id_count"})
summary_count.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/Experian_Control_Test/Experian_LR_Summary_Segments.csv",index=False)

Eperian_in_Q4_Control.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/Experian_Control_Test/Experian_in_Q4_Segments.csv",index=False)

In [15]:
summary_count

,segment,id_count
0,Experian_C_Holiday Shoppers,45109
1,Experian_C_Home Focused,108612
2,Experian_C_Rising Hispanics,110393
3,Experian_C_Spending Fanatics,47445
4,Experian_C_Urban Affluence,88390
5,Experian_T_Holiday Shoppers,857058
6,Experian_T_Home Focused,2063612
7,Experian_T_Rising Hispanics,2097462
8,Experian_T_Spending Fanatics,901449
9,Experian_T_Urban Affluence,1679401
